In [182]:
# SZÓTÁR NEM KÉZZEL CSINÁLVA

In [183]:
import pandas as pd

In [184]:
xls = pd.ExcelFile('szotar_2.1.xlsx')

In [185]:
dfs=[pd.read_excel(xls,sheet) for i, sheet in enumerate(xls.sheet_names)]

In [186]:
filtereddfs=\
    [df[pd.concat([df[column].astype(str).str.contains('Mom',na=True, case=False)
                   for column in df.columns],axis=1).any(axis=1)]
                   for df in dfs]

In [187]:
names = \
[name.replace(u'\xa0', u' ').replace(' ','|').replace('|',' ').strip()
 for namelist in [df['name_list'].tolist()
 for df in filtereddfs]
 for name in namelist]

In [188]:
names[3]

'Körömi Attila'

In [189]:
autodict = set(names)

In [190]:
# SZÓTÁR KÉZZEL CSINÁLVA

In [191]:
kezidict = set([
'Fekete-Győr András',
'Borbélyné Bárdi Zsuzsanna',
'Hajnal Miklós',
'Körömi Attila',
'Mándi László',
'Orosz Anna',
'Tóth Endre',
'Cseh Katalin',
'Donáth Anna Júlia',
'Soproni Tamás',
'Déri Tibor',
'Orosz Anna',
'Nyirati Klára',
'Pikó András',
'Balogh Csaba',
'Kerpel-Fronius Gábor',
'Momentum',
'momentum',
'Fekete-Győr'
'Fekete Győr',
'Donáth Anna'
'Donáth',
'Nyirati',
'Pikó',
'Kerpel-Fronius',
'Mándi',
'Körömi',
'Kádár Barnabás',
'Hadházy',
'Hadházy Ákos'
'Szél Bernadett',
])

In [192]:
autodict-kezidict

{'Berg Dániel'}

In [193]:
kezidict-autodict

{'Balogh Csaba',
 'Donáth AnnaDonáth',
 'Déri Tibor',
 'Fekete-GyőrFekete Győr',
 'Hadházy',
 'Hadházy ÁkosSzél Bernadett',
 'Kerpel-Fronius',
 'Kerpel-Fronius Gábor',
 'Kádár Barnabás',
 'Körömi',
 'Momentum',
 'Mándi',
 'Nyirati',
 'Nyirati Klára',
 'Pikó',
 'Pikó András',
 'Soproni Tamás',
 'momentum'}

In [194]:
momentum_searchlist = [[each] for each in list(kezidict)]

In [195]:
import glob
import datetime
import glob
import stanza
import pandas as pd
import time
import sys

In [196]:
def filetime(file):
    year=file.split('/')[-1][5:9]
    month=file.split('/')[-1][10:12]
    day=file.split('/')[-1][13:15]
    hour=file.split('/')[-1][16:18]
    return [int(each) for each in [year, month, day, hour]]

In [197]:
stanza.download('hu')
nlp = stanza.Pipeline('hu')

2020-07-26 14:43:00 INFO: Downloading default packages for language: hu (Hungarian)...
2020-07-26 14:43:01 INFO: File exists: /mnt/volume/jupyter/stanza_resources/hu/default.zip.
2020-07-26 14:43:06 INFO: Finished downloading models and saved to /mnt/volume/jupyter/stanza_resources.
2020-07-26 14:43:06 INFO: Loading these models for language: hu (Hungarian):
| Processor | Package |
-----------------------
| tokenize  | szeged  |
| pos       | szeged  |
| lemma     | szeged  |
| depparse  | szeged  |

2020-07-26 14:43:06 INFO: Use device: cpu
2020-07-26 14:43:06 INFO: Loading: tokenize
2020-07-26 14:43:06 INFO: Loading: pos
2020-07-26 14:43:08 INFO: Loading: lemma
2020-07-26 14:43:08 INFO: Loading: depparse
2020-07-26 14:43:10 INFO: Done loading processors!


In [198]:
def onlywithneighbours(ofthislist):
    try:
        filtered = [each for each in ofthislist 
                    if each+1 in ofthislist or each-1 in ofthislist]
    except TypeError:
        filtered = [each for each in ofthislist 
                    if str(int(each)+1) in ofthislist or str(int(each)-1) in ofthislist]
    return filtered

In [199]:
def split_into_numerical_sequences(inlist):

    inlist=sorted(inlist)

    breakindeces=[i for i,j in enumerate(inlist)
                    if (j+1 not in inlist and j in inlist)]

    sublists=[]
    for index, each in enumerate(breakindeces):
        if index==0:
            sublists.append([x for x in inlist
                               if x<=inlist[each]])
        if index!=0:
            sublists.append([x for x in inlist
                               if x<=inlist[each] and x>inlist[breakindeces[index-1]]])

    return sublists

In [200]:
def validalias(alias):
    return True
   # if len(str(alias).strip()) > 5 and len(str(alias).strip().split(' ')) > 1: return True
   # else: return False

In [201]:
def matchfinder(text,searchforthese):
    matches=[alias
             for persondata_searchtarget in searchforthese
             for alias in persondata_searchtarget
             if validalias(alias) and alias.lower() in str(text).lower()]
    return matches

In [202]:
def fillextracols(whichdictionary, whichrowtofill, withthislist, unlessitslongerthanthis):
    if not len(withthislist) > unlessitslongerthanthis and len(withthislist)>0:
        for i, e in enumerate(withthislist):
            targetdf.loc[whichrowtofill,whichdictionary+str(i)]=e

In [203]:
def prepare_extra_columns(num_of_columns_for_each_dict):
    for each in num_of_columns_for_each_dict:
        for index in range(num_of_columns_for_each_dict[each]):
            targetdf[each + str(index)]=''

In [204]:
class dictionary_class:
    def __init__(self, name, maxcolnum, searchlist=None, geo=False):
        self.name = name
        self.maxcolnum = maxcolnum
        self.searchlist = searchlist
        self.geo = geo

In [205]:
dictionaries=[
dictionary_class('momentumdict',
           10,
           momentum_searchlist)]

In [206]:
def get_files_sorted_by_date_after_a_date(look_for_this_pattern,cutoffdate):
    
    csvs = glob.glob(look_for_this_pattern)
    datetimes=[datetime.datetime(int(each.split('/')[-1][5:9]),
                             int(each.split('/')[-1][10:12]),
                             int(each.split('/')[-1][13:15]),
                             int(each.split('/')[-1][16:18]),
                             int(each.split('/')[-1][19:21]),
                             int(each.split('/')[-1][22:24])) for each in csvs]

    dt_csvs_filtered=[[dt, csv] for dt, csv in zip(datetimes,csvs) if dt > datetime.datetime(*cutoffdate)]
    
    sorted_filtered_csvs = [csv
                        for _, csv in sorted(
                                         zip([eachpair[0] for eachpair in dt_csvs_filtered],
                                             [eachpair[1] for eachpair in dt_csvs_filtered]))]
    
    return sorted_filtered_csvs

In [207]:
len(get_files_sorted_by_date_after_a_date('/mnt/volume/anagy/mediascraper/mediaScraper/output/data*csv',
                                      [2020,7,1,0]))

311

In [208]:

debugmode = False
wildcard = '/mnt/volume/anagy/mediascraper/mediaScraper/output/data*csv'

todofiles=get_files_sorted_by_date_after_a_date('/mnt/volume/anagy/mediascraper/mediaScraper/output/data*csv',
                                               [2020,7,1,0])
while len(todofiles) != 0:
    print('processing: '+todofiles[0])    
    
    targetcsv=todofiles[0]
    try:
        targetdf = pd.read_csv(targetcsv)

        prepare_extra_columns({dictionary.name: dictionary.maxcolnum for dictionary in dictionaries})

        cells = list(targetdf['TEXT'])

        start_time = time.time()
        for idictionary, dictionary in enumerate(dictionaries):
            if idictionary==7 or not debugmode:
            #if dictionary.name == 'instit_dict':
                for icell, cell in enumerate(cells):
                    if type(cell) is not float:
                        if icell < 10 or not debugmode:
                            if icell%1000==0: print(icell)
                            if dictionary.searchlist is not None:
                                fillextracols(dictionary.name,icell,matchfinder(cell,dictionary.searchlist),dictionary.maxcolnum)
                                #print('hw')
                            if dictionary.searchlist is None:
                                fillextracols(dictionary.name,icell,stanzanamesearch(cell),dictionary.maxcolnum)
        end_time = time.time()
        print("--- %s seconds ---" % (time.time() - start_time))
        if not debugmode: targetdf.to_csv('/mnt/volume/jupyter/szokereso/resultfiles/onlymomentum/updated3_dict_onlymomentum_'+targetcsv.split('/')[-1].split('.')[0]+'_szokereso_result.csv')
        print(targetcsv)
    except KeyboardInterrupt:
        print('KeyboardInterrupt')
        break
    except:
        the_type, the_value, the_traceback = sys.exc_info()
        outlist = [the_type, the_value, targetcsv, dictionary.name, icell]
        if not debugmode:
            with open('/mnt/volume/jupyter/szokereso/resultfiles/onlymomentum/updated3_dict_onlymomentum_'+targetcsv.split('/')[-1].split('.')[0]+'_ERRORLOG.txt', 'w') as f:
                for item in outlist:
                    f.write("%s\n" % item)
                
    todofiles=get_files_sorted_by_date_after_a_date('/mnt/volume/anagy/mediascraper/mediaScraper/output/data*csv',
                                                     filetime(todofiles[0])[:3]+[filetime(todofiles[0])[-1]+1]) # hour increased by 1



processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-01_00:00:24.csv
0
1000
2000
3000
--- 1.1955897808074951 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-01_00:00:24.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-01_02:00:18.csv
0
1000
2000
3000
--- 1.4980616569519043 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-01_02:00:18.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-01_04:00:22.csv
0
1000
2000
3000
--- 1.159391164779663 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-01_04:00:22.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-01_06:00:26.csv
0
1000
2000
3000
--- 1.3633496761322021 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-01_06:00:26.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-01_08:01:18.csv
1000
2000
300

1000
2000
3000
--- 1.3285696506500244 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-03_22:01:05.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-04_00:00:29.csv
0
1000
2000
3000
--- 1.0456664562225342 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-04_00:00:29.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-04_02:00:24.csv
0
1000
2000
3000
--- 1.1490981578826904 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-04_02:00:24.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-04_04:00:24.csv
0
1000
2000
3000
--- 1.351499319076538 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-04_04:00:24.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-04_06:00:25.csv
0
1000
2000
3000
--- 1.2350196838378906 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data

1000
2000
3000
--- 1.5788781642913818 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-07_00:00:40.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-07_02:00:22.csv
0
1000
2000
3000
--- 1.4071950912475586 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-07_02:00:22.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-07_04:00:24.csv
0
1000
2000
3000
--- 1.7230439186096191 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-07_04:00:24.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-07_06:00:28.csv
0
1000
2000
3000
--- 1.4489059448242188 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-07_06:00:28.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-07_08:01:08.csv
0
1000
2000
3000
--- 1.6266767978668213 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/dat

1000
2000
3000
--- 1.7594959735870361 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-10_02:00:22.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-10_04:00:26.csv
0
1000
2000
3000
--- 1.864612340927124 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-10_04:00:26.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-10_06:00:31.csv
0
1000
2000
3000
--- 2.0759613513946533 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-10_06:00:31.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-10_08:01:10.csv
0
1000
2000
3000
--- 2.0042357444763184 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-10_08:01:10.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-10_10:01:48.csv
0
1000
2000
3000
--- 1.8414807319641113 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data

0
1000
2000
3000
--- 1.9761133193969727 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-13_04:00:26.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-13_06:00:26.csv
0
1000
2000
3000
--- 1.9235291481018066 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-13_06:00:26.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-13_08:01:14.csv
0
1000
2000
3000
--- 2.199394702911377 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-13_08:01:14.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-13_10:01:58.csv
0
1000
2000
3000
--- 1.9492876529693604 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-13_10:01:58.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-13_12:02:41.csv
0
1000
2000
3000
--- 1.874192714691162 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/dat

0
1000
2000
3000
--- 2.1753807067871094 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-16_06:00:28.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-16_08:01:07.csv
0
1000
2000
3000
--- 2.41702938079834 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-16_08:01:07.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-16_10:01:53.csv
0
1000
2000
3000
--- 2.251373767852783 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-16_10:01:53.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-16_12:02:31.csv
0
1000
2000
3000
--- 2.045072078704834 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-16_12:02:31.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-16_14:01:53.csv
0
1000
2000
3000
--- 2.499448299407959 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2

0
1000
2000
3000
--- 2.464597702026367 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-19_10:01:14.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-19_12:01:25.csv
0
1000
2000
3000
--- 2.2368991374969482 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-19_12:01:25.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-19_14:01:05.csv
0
1000
2000
3000
--- 2.119530439376831 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-19_14:01:05.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-19_16:01:09.csv
0
1000
2000
3000
--- 2.4037623405456543 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-19_16:01:09.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-19_18:01:05.csv
0
1000
2000
3000
--- 2.215374231338501 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data

1000
2000
3000
--- 1.8371918201446533 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-22_12:02:17.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-22_14:01:30.csv
0
1000
2000
3000
--- 1.8880677223205566 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-22_14:01:30.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-22_16:01:47.csv
0
1000
2000
3000
--- 2.2644472122192383 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-22_16:01:47.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-22_18:01:28.csv
0
1000
2000
3000
--- 1.9996709823608398 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-22_18:01:28.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-22_20:01:08.csv
0
1000
2000
3000
--- 2.0832126140594482 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/dat

0
1000
2000
3000
--- 2.305959701538086 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-25_14:00:50.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-25_16:00:45.csv
0
1000
2000
3000
--- 2.2799012660980225 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-25_16:00:45.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-25_18:01:13.csv
0
1000
2000
3000
--- 2.139977216720581 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-25_18:01:13.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-25_20:00:49.csv
0
1000
2000
3000
--- 2.414958953857422 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-25_20:00:49.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-25_22:00:47.csv
0
1000
2000
3000
--- 2.317941188812256 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_

In [216]:
onlymomFiles = list(
               set(glob.glob('resultfiles/onlymomentum/updated3_dict_onlymomentum_*2020-07-*')) \
              -set(glob.glob('resultfiles/onlymomentum/updated3_dict_onlymomentum_*2020-07-25*')) \
              -set(glob.glob('resultfiles/onlymomentum/updated3_dict_onlymomentum_*2020-07-26*')) \
              -set(glob.glob('resultfiles/onlymomentum/updated3_dict_onlymomentum_*2020-07-27*')))

In [217]:
onlymomFiles

['resultfiles/onlymomentum/updated3_dict_onlymomentum_data_2020-07-06_14:02:23_szokereso_result.csv',
 'resultfiles/onlymomentum/updated3_dict_onlymomentum_data_2020-07-16_00:00:31_szokereso_result.csv',
 'resultfiles/onlymomentum/updated3_dict_onlymomentum_data_2020-07-23_14:01:27_szokereso_result.csv',
 'resultfiles/onlymomentum/updated3_dict_onlymomentum_data_2020-07-11_02:00:25_szokereso_result.csv',
 'resultfiles/onlymomentum/updated3_dict_onlymomentum_data_2020-07-07_04:00:24_szokereso_result.csv',
 'resultfiles/onlymomentum/updated3_dict_onlymomentum_data_2020-07-03_14:03:28_szokereso_result.csv',
 'resultfiles/onlymomentum/updated3_dict_onlymomentum_data_2020-07-08_14:01:51_szokereso_result.csv',
 'resultfiles/onlymomentum/updated3_dict_onlymomentum_data_2020-07-08_20:01:26_szokereso_result.csv',
 'resultfiles/onlymomentum/updated3_dict_onlymomentum_data_2020-07-12_10:01:01_szokereso_result.csv',
 'resultfiles/onlymomentum/updated3_dict_onlymomentum_data_2020-07-06_04:00:22_szo

In [218]:
dflist=[]
for eachmomfile in onlymomFiles:
    df=pd.read_csv(eachmomfile)
    dflist.append(df[~df['momentumdict0'].isna()])

In [219]:
len(dflist)

289

In [220]:
dfUnifiedNanFiltered = pd.concat([df for df in dflist])

In [221]:
dfUnifiedNanFiltered.to_csv('dfUnifiedNanFilteredUpdated2.csv')

KeyboardInterrupt: 

In [222]:
dfUnifiedNanFiltered[['DOC_ID','TITLE','SCRAPETIME','TEXT','momentumdict0', 'momentumdict1',
                      'momentumdict2', 'momentumdict3', 'momentumdict4', 'momentumdict5',
                      'momentumdict6', 'momentumdict7', 'momentumdict8', 'momentumdict9']].to_csv('dfUnifiedNanFilteredOnlySomeColsUpdated3.csv')

In [224]:
!pwd

/mnt/volume/jupyter/szokereso


In [223]:
dfUnifiedNanFiltered[['DOC_ID','TITLE','SCRAPETIME','TEXT','momentumdict0', 'momentumdict1',
                      'momentumdict2', 'momentumdict3', 'momentumdict4', 'momentumdict5',
                      'momentumdict6', 'momentumdict7', 'momentumdict8', 'momentumdict9']]

,DOC_ID,TITLE,SCRAPETIME,TEXT,momentumdict0,momentumdict1,momentumdict2,momentumdict3,momentumdict4,momentumdict5,momentumdict6,momentumdict7,momentumdict8,momentumdict9
164,165,A koronavírus anyagilag is megviselte a lakoss...,2020-07-06 14:00:17,"\n\nA megkérdezettek 13 százalékának kisebb, k...",momentum,Momentum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263,264,Szatmáry Kristóf nem panaszkodhat: szépen gyar...,2020-07-06 14:00:17,\n\nA fideszes politikus jó érzékkel szállt be...,momentum,Momentum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277,278,Hogyan birkóztak meg a közép- és dél-kelet eur...,2020-07-06 14:00:17,"\n\nGazdasági krízis, politikai nyomás, az inf...",momentum,Momentum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,308,Leváltotta a Fidesszel szövetkező alpolgármest...,2020-07-06 14:00:17,\n\nSzerinte az LMP-s és DK-s alpolgármesterek...,Balogh Csaba,momentum,Momentum,NaN,NaN,NaN,NaN,NaN,NaN,NaN
331,332,Baranyi Krisztinát ábrázoló ostoba szexista gú...,2020-07-06 14:00:17,\n\nBácskai János eddig sem jó modoráról volt ...,momentum,Momentum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2926,Müller Cecília még mindig nem hajlandó találko...,2020-07-03 18:00:17,\n\nAz országos tiszti főorvos korábban arra h...,Hadházy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3040,3041,Leváltotta a Fidesszel összefogó alpolgármeste...,2020-07-03 18:00:17,\n\nMiután Göd momentumos polgármester a hétfő...,Balogh Csaba,momentum,Momentum,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3071,3072,Baranyi Krisztinát ábrázoló primitív szexista ...,2020-07-03 18:00:17,"\n\nMátyás Ferenc, ferencvárosi momentumos kép...",momentum,Momentum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3094,3095,A lakók feje fölött döntött egy utca leburkolá...,2020-07-03 18:00:17,\n\n * Nagyobb részt állami támogatásból szil...,momentum,Momentum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [127]:
numberOfUniqueTitles = len(dfUnifiedNanFiltered['TITLE'].drop_duplicates())

In [128]:
numberOfTitles       = len(dfUnifiedNanFiltered['TITLE'])

In [138]:
dfUnifiedNanFiltered.groupby('OUTLET').agg(['count'])['TITLE']

,count
OUTLET,
168ora.hu,28
168óra,1062
24.hu,3590
444.hu,946
888.hu,1679
ATV,861
Alfahír,888
Blikk,133
Blikk Rúzs,45


In [137]:
dfUnifiedNanFiltered.groupby('OUTLET').agg(['count'])['TITLE'].to_csv('OutletCount.csv')

In [130]:
numberOfTitles

34292

In [116]:
[each for each dfUnifiedNanFiltered.loc[1500,'TEXT']]

SyntaxError: invalid syntax (<ipython-input-116-0a109465fe4d>, line 1)

In [120]:
[each for each in dfUnifiedNanFiltered.loc[1500]['TEXT']][0]

'\n\n# Zac Efron tökéletes aputeste elviszi az új világjáró sorozatát\n\n  * __\n\nMészáros Júlia\n\nújságíró.  2020\\. 07. 19. 09:45\n\nLegalább a vártnál is több emberhez jut el a kultúra és a fenntarthatóság.\n\nNemrég mutatta be a Netflix a _Down to Earth_ című dokusorozatot, amiben **Zac\nEfron** utazza körbe a világot, hogy megmutasson különböző kultúrákat, illetve\nazt, hogy hol miként kezelik a környezeti fenntarthatóságot. A New York Post\narról ír, hogy a téma nem feltétlenül mozgatna meg tömegeket, ha nem a – most\népp szakállas – Zac Efron mutatná be, és ha nem vetkőzne félmeztelenre\ntöbbször is.\n\nA nézők közül többen rögtön daddynek – angol szleng a borzalmasan vonzó\nidősebb férfiakra – nevezték Efront, tele a Twitter a színész aputeste után\nvágyakozók posztjaival. Volt aki azt írta, szeretné megszerezni Efront\nszüleinek telefonszámát, hogy hagyhasson egy háromperces hangüzenetet, amiben\nujjongva gratulál.\n\nMost pedig következzen a lényeg, Zac Efron pár évvel ezel